In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Acute_Myeloid_Leukemia"
cohort = "GSE222124"

# Input paths
in_trait_dir = "../../input/GEO/Acute_Myeloid_Leukemia"
in_cohort_dir = "../../input/GEO/Acute_Myeloid_Leukemia/GSE222124"

# Output paths
out_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/GSE222124.csv"
out_gene_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE222124.csv"
out_clinical_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE222124.csv"
json_path = "../../output/preprocess/Acute_Myeloid_Leukemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression alterations in the cell lines Jurkat, THP1, and KHYG-1 upon treatment with glycodelin from non-small-cell lung cancer cell line supernatant"
!Series_summary	"The function of glycodelin A in the endometrium, during the menstrual cycle, and before or during pregnancy is well characterized. Numerous studies have investigated the highly pleiotropic effects and the modulation of different leukocytes upon glycodelin interaction.  In NSCLC, the function of glycodelin is not known, yet. While it was shown to be highly expressed in lung tumor tissue compared to normal lung, the functionality and possible immunomodulating characteristics remain to be described."
!Series_overall_design	"The three different immune cell lines Jurkat, THP1, and KHYG-1 were treated with glycodelin containing cell culture supernatant derived from a NSCLC cell line. Differential expression upon glycodelin treatment was assessed and further pathway analyses were per

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# The dataset contains gene expression data for immune cell lines, so set to True
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (leukemia): 
# Using the cell line information as trait indicator at key 0
# There are different leukemia types: T cell leukemia, Acute monocytic leukemia, NK cell leukemia
trait_row = 0  

# Age and gender not available in the data
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert leukemia cell type to binary classification for AML"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if it's Acute monocytic leukemia (AML)
    if 'monocytic leukemia' in value.lower() or 'aml' in value.lower():
        return 1  # Positive for AML
    else:
        return 0  # Not AML (T cell leukemia or NK cell leukemia)

def convert_age(value):
    """Convert age to float (not used but defined for completeness)"""
    return None  # Age data not available

def convert_gender(value):
    """Convert gender to binary (not used but defined for completeness)"""
    return None  # Gender data not available

# 3. Save Metadata - Initial filtering
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with feature extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    preview = preview_df(clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'GSM6915207': [0.0], 'GSM6915208': [0.0], 'GSM6915209': [0.0], 'GSM6915210': [0.0], 'GSM6915211': [0.0], 'GSM6915212': [0.0], 'GSM6915213': [0.0], 'GSM6915214': [0.0], 'GSM6915215': [0.0], 'GSM6915216': [0.0], 'GSM6915217': [0.0], 'GSM6915218': [0.0], 'GSM6915219': [0.0], 'GSM6915220': [0.0], 'GSM6915221': [0.0], 'GSM6915222': [0.0], 'GSM6915223': [0.0], 'GSM6915224': [0.0], 'GSM6915225': [0.0], 'GSM6915226': [0.0], 'GSM6915227': [0.0], 'GSM6915228': [0.0], 'GSM6915229': [0.0], 'GSM6915230': [0.0], 'GSM6915231': [1.0], 'GSM6915232': [1.0], 'GSM6915233': [1.0], 'GSM6915234': [1.0], 'GSM6915235': [1.0], 'GSM6915236': [1.0], 'GSM6915237': [1.0], 'GSM6915238': [1.0], 'GSM6915239': [1.0], 'GSM6915240': [1.0], 'GSM6915241': [1.0], 'GSM6915242': [1.0], 'GSM6915243': [1.0], 'GSM6915244': [1.0], 'GSM6915245': [1.0], 'GSM6915246': [1.0], 'GSM6915247': [1.0], 'GSM6915248': [1.0], 'GSM6915249': [1.0], 'GSM6915250': [1.0], 'GSM6915251': [1.0], 'GSM6915252': [1.0], 'GSM691525

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous output
# These appear to be Affymetrix probe IDs (e.g., "1007_s_at", "1053_at")
# They are not standard human gene symbols, which typically look like BRCA1, TP53, etc.
# Affymetrix IDs need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in the gene annotation data
# From previous output, we see:
# - The gene expression data uses "ID" as identifiers ("1007_s_at", "1053_at", etc.)
# - In the gene annotation, "ID" column contains these same identifiers
# - The "Gene Symbol" column contains the corresponding gene symbols

# 2. Get gene mapping dataframe
prob_col = "ID"  # Column with the probe IDs
gene_col = "Gene Symbol"  # Column with the gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Preview the mapping
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene-level data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)

# Preview the result of gene mapping
print("\nGene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(gene_data.index[:20])  # First 20 gene symbols


Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}



Gene expression data after mapping:
Shape: (21278, 70)
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2',
       'AADACP1', 'AADAT', 'AAED1', 'AAGAB', 'AAK1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since the clinical data was already properly processed in step 2, let's load that file instead of redefining
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Loaded clinical data from {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(clinical_df))

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Verify that the trait column has at least two unique values
unique_trait_values = linked_data[trait].unique()
print(f"Unique values in trait column: {unique_trait_values}")

# 5. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
note = "Dataset contains different leukemia cell lines, with AML (acute monocytic leukemia) as the trait of interest."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=note
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Processed dataset saved to {out_data_file}")
else:
    print("Dataset not usable due to bias in trait distribution. Data not saved.")

Normalized gene data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE222124.csv
Loaded clinical data from ../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE222124.csv
Clinical data shape: (1, 70)
Clinical data preview:
{'GSM6915207': [0.0], 'GSM6915208': [0.0], 'GSM6915209': [0.0], 'GSM6915210': [0.0], 'GSM6915211': [0.0], 'GSM6915212': [0.0], 'GSM6915213': [0.0], 'GSM6915214': [0.0], 'GSM6915215': [0.0], 'GSM6915216': [0.0], 'GSM6915217': [0.0], 'GSM6915218': [0.0], 'GSM6915219': [0.0], 'GSM6915220': [0.0], 'GSM6915221': [0.0], 'GSM6915222': [0.0], 'GSM6915223': [0.0], 'GSM6915224': [0.0], 'GSM6915225': [0.0], 'GSM6915226': [0.0], 'GSM6915227': [0.0], 'GSM6915228': [0.0], 'GSM6915229': [0.0], 'GSM6915230': [0.0], 'GSM6915231': [1.0], 'GSM6915232': [1.0], 'GSM6915233': [1.0], 'GSM6915234': [1.0], 'GSM6915235': [1.0], 'GSM6915236': [1.0], 'GSM6915237': [1.0], 'GSM6915238': [1.0], 'GSM6915239': [1.0], 'GSM6915240': [1.0], 'GSM6915241': [1.0], 'GSM691524

Data shape after handling missing values: (70, 19846)
Unique values in trait column: [0. 1.]
For the feature 'Acute_Myeloid_Leukemia', the least common label is '1.0' with 24 occurrences. This represents 34.29% of the dataset.
The distribution of the feature 'Acute_Myeloid_Leukemia' in this dataset is fine.



Processed dataset saved to ../../output/preprocess/Acute_Myeloid_Leukemia/GSE222124.csv
